# RTW MAM Model V1 
# County Data First

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
import pickle

In [2]:
#SET DATE FILTERS & FileName

file = 'CountyMAMV3_629.csv'
state_file = 'State_MAM_629.csv'

preval_start = '2020-06-15'
preval_end = '2020-06-28'

curr_start = '2020-06-22'
curr_end = '2020-06-28'

prev_start = '2020-06-15'
prev_end = '2020-06-21'

curr_test_start = [int(20200628)]
curr_test_end = [int(20200622)]
prev_test_start = [int(20200621)]
prev_test_end = [int(20200615)]

In [3]:
#UPDATES NEEDED - bring in base file, within base file add in state 2 digit codes
#Need to add in the NYC FIPS dummy code

#Bring in County Data set
nytimes = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
counties = pd.read_csv(nytimes,dtype={'fips': str})
print(counties.head())



base = "https://raw.githubusercontent.com/cbmeco/COVID_MAM/master/base_rtw_v1.csv"
pop_df = pd.read_csv(base,
                dtype={'fips': str})

state_base = "https://raw.githubusercontent.com/cbmeco/COVID_MAM/master/state_base.csv"
state_pop_df = pd.read_csv(state_base)
print(state_pop_df.head())

#pop_df = pd.read_csv("base_rtw_v1.csv",
#                dtype={'fips': str})

#base['fips2'] = np.where(base['county']=='New York City','36067',base['fips'])

counties.loc[counties['county'] == 'New York City', 'fips'] = '36067'


print(counties.head())

         date     county       state   fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061      1       0
1  2020-01-22  Snohomish  Washington  53061      1       0
2  2020-01-23  Snohomish  Washington  53061      1       0
3  2020-01-24       Cook    Illinois  17031      1       0
4  2020-01-24  Snohomish  Washington  53061      1       0
        State state_code  Comcast Employees  state_pop  CAE  Cable Stores  \
0     Alabama         AL                407    4908621   65            40   
1      Alaska         AK                  0    3038999    0             0   
2     Arizona         AZ                744    7378494  237            17   
3    Arkansas         AR                117    3038999    1            13   
4  California         CA               5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  
0                  9       178              1  
1                  0         0              0  
2                  5        87         

# Create Active Case Count

In [4]:
#Create new cases field
#Sort df by Fips and Date
counties.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(counties.head())

#Take difference between rows for new case and new death numbers
counties['case_diff'] = counties.cases.diff()
counties['death_diff'] = counties.deaths.diff()


print(counties.head())

             date   county    state   fips  cases  deaths
9480   2020-03-24  Autauga  Alabama  01001      1       0
10835  2020-03-25  Autauga  Alabama  01001      4       0
12367  2020-03-26  Autauga  Alabama  01001      6       0
14025  2020-03-27  Autauga  Alabama  01001      6       0
15803  2020-03-28  Autauga  Alabama  01001      6       0
             date   county    state   fips  cases  deaths  case_diff  \
9480   2020-03-24  Autauga  Alabama  01001      1       0        NaN   
10835  2020-03-25  Autauga  Alabama  01001      4       0        3.0   
12367  2020-03-26  Autauga  Alabama  01001      6       0        2.0   
14025  2020-03-27  Autauga  Alabama  01001      6       0        0.0   
15803  2020-03-28  Autauga  Alabama  01001      6       0        0.0   

       death_diff  
9480          NaN  
10835         0.0  
12367         0.0  
14025         0.0  
15803         0.0  


In [5]:
#Filter for past 14 days
#range_14 = counties[counties['date']>=preval_start]

range_14 = counties[(counties['date']>=preval_start) & (counties['date']<=preval_end)]

print(range_14.head(15))

              date   county    state   fips  cases  deaths  case_diff  \
236775  2020-06-15  Autauga  Alabama  01001    368       6        7.0   
239806  2020-06-16  Autauga  Alabama  01001    373       7        5.0   
242838  2020-06-17  Autauga  Alabama  01001    375       7        2.0   
245869  2020-06-18  Autauga  Alabama  01001    400       8       25.0   
248903  2020-06-19  Autauga  Alabama  01001    411       8       11.0   
251941  2020-06-20  Autauga  Alabama  01001    431       9       20.0   
254987  2020-06-21  Autauga  Alabama  01001    434       9        3.0   
258033  2020-06-22  Autauga  Alabama  01001    442       9        8.0   
261081  2020-06-23  Autauga  Alabama  01001    453       9       11.0   
264131  2020-06-24  Autauga  Alabama  01001    469      11       16.0   
267185  2020-06-25  Autauga  Alabama  01001    479      11       10.0   
270245  2020-06-26  Autauga  Alabama  01001    488      11        9.0   
273314  2020-06-27  Autauga  Alabama  01001    498 

In [6]:
#Group by FIPS, sum up new Cases for prevalence - for 7 day averages will take mean after filtering for 7 days
prevalence_grouped = range_14.groupby(['fips'], as_index=False).sum()

print(prevalence_grouped.head(25))

     fips  cases  deaths  case_diff  death_diff
0   01001   6124     129      142.0         6.0
1   01003   6301     128      196.0         1.0
2   01005   3903      14       82.0         0.0
3   01007   1875      14       55.0         0.0
4   01009   2138      14       77.0         0.0
5   01011   4636     141       54.0         2.0
6   01013   8059     366       63.0         2.0
7   01015   3072      61       42.0         2.0
8   01017   7326     375       94.0         1.0
9   01019    833      86       17.0         2.0
10  01021   2389      39       65.0         1.0
11  01023   2589     163       13.0         1.0
12  01025   3108      59       68.0         2.0
13  01027    454      28        5.0         0.0
14  01029    319      14        4.0         0.0
15  01031   4466      14       58.0         0.0
16  01033   4343      70       96.0         0.0
17  01035   1944      24       70.0         2.0
18  01037    704      14        9.0         0.0
19  01039   3315      35      132.0     

In [7]:
#Join in Prevalence to 
prevalence_w_pop = pd.merge(pop_df,
                 prevalence_grouped[['fips','case_diff']],
                 on='fips', 
                how='left')
print(prevalence_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.

In [8]:
prevalence_w_pop['active_cases_100k'] = ((prevalence_w_pop['case_diff']/prevalence_w_pop['pop'])*100000)
print(prevalence_w_pop.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.0         201.0              146.0     3640.0   
3                0.0    0.0           0.0                0.0        0.0   
4                0.0    0.0           0.0                3.0        9.0   

        Geo Area %Retail Closed in County Updated 5/26      Division  \
0            NaN                                   NaN     Northeast

# Create 7 Day Rolling Avgs for Cases & Deaths

In [9]:
#Filter Diff datasets for current and previous 7 day periods

#join prev and current together with pop

range_current7 = counties[(counties['date']>=curr_start) & (counties['date']<=curr_end)]

range_prev7 = counties[(counties['date']>=prev_start) & (counties['date']<=prev_end)]

print(range_prev7.head(10))

              date   county    state   fips  cases  deaths  case_diff  \
236775  2020-06-15  Autauga  Alabama  01001    368       6        7.0   
239806  2020-06-16  Autauga  Alabama  01001    373       7        5.0   
242838  2020-06-17  Autauga  Alabama  01001    375       7        2.0   
245869  2020-06-18  Autauga  Alabama  01001    400       8       25.0   
248903  2020-06-19  Autauga  Alabama  01001    411       8       11.0   
251941  2020-06-20  Autauga  Alabama  01001    431       9       20.0   
254987  2020-06-21  Autauga  Alabama  01001    434       9        3.0   
236776  2020-06-15  Baldwin  Alabama  01003    383       9        4.0   
239807  2020-06-16  Baldwin  Alabama  01003    389       9        6.0   
242839  2020-06-17  Baldwin  Alabama  01003    392       9        3.0   

        death_diff  
236775         0.0  
239806         1.0  
242838         0.0  
245869         1.0  
248903         0.0  
251941         1.0  
254987         0.0  
236776         0.0  
239807 

In [10]:
#Group by with mean
current_grouped = range_current7.groupby(['fips'],as_index=False).mean()
previous_grouped = range_prev7.groupby(['fips'], as_index=False).mean()

current_grouped['curr7_case'] = current_grouped['case_diff']
current_grouped['curr7_death'] = current_grouped['death_diff']
previous_grouped['prev7_case'] = previous_grouped['case_diff']
previous_grouped['prev7_death'] = previous_grouped['death_diff']
print(current_grouped.head())
print(previous_grouped.head())

    fips       cases     deaths  case_diff  death_diff  curr7_case  \
0  01001  476.000000  10.714286   9.857143    0.428571    9.857143   
1  01003  496.142857   9.285714  20.714286    0.142857   20.714286   
2  01005  299.428571   1.000000   6.428571    0.000000    6.428571   
3  01007  147.142857   1.000000   5.000000    0.000000    5.000000   
4  01009  173.857143   1.000000   5.714286    0.000000    5.714286   

   curr7_death  
0     0.428571  
1     0.142857  
2     0.000000  
3     0.000000  
4     0.000000  
    fips       cases    deaths  case_diff  death_diff  prev7_case  prev7_death
0  01001  398.857143  7.714286  10.428571    0.428571   10.428571     0.428571
1  01003  404.000000  9.000000   7.285714    0.000000    7.285714     0.000000
2  01005  258.142857  1.000000   5.285714    0.000000    5.285714     0.000000
3  01007  120.714286  1.000000   2.857143    0.000000    2.857143     0.000000
4  01009  131.571429  1.000000   5.285714    0.000000    5.285714     0.000000


In [11]:
#Merge both sets with population
current_w_pop = pd.merge(pop_df,
                 current_grouped[['fips','curr7_case', 'curr7_death']],
                 on='fips', 
                how='left')
print(current_w_pop.head(10))

previous_w_pop = pd.merge(pop_df,
                 previous_grouped[['fips','prev7_case', 'prev7_death']],
                 on='fips', 
                how='left')
print(previous_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.

In [12]:
current_w_pop['cur_7rollavg_cases'] = ((current_w_pop['curr7_case']/current_w_pop['pop'])*100000)
current_w_pop['cur_7rollavg_deaths'] = ((current_w_pop['curr7_death']/current_w_pop['pop'])*100000)

In [13]:
previous_w_pop['prev_7rollavg_cases'] = ((previous_w_pop['prev7_case']/previous_w_pop['pop'])*100000)
previous_w_pop['prev_7rollavg_deaths'] = ((previous_w_pop['prev7_death']/previous_w_pop['pop'])*100000)

In [14]:
#Merge with prevalence dataset
prev_curr = pd.merge(prevalence_w_pop,
                 current_w_pop[['fips','cur_7rollavg_cases', 'cur_7rollavg_deaths','curr7_case','curr7_death']],
                 on='fips', 
                how='left')
print(prev_curr.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...       \
0                0.0    0.0           0.0                0.0     ...        
1                0.0    0.0           0.0                2.0     ...        
2              965.0

In [15]:
county_set = pd.merge(prev_curr,
                 previous_w_pop[['fips','prev_7rollavg_cases', 'prev_7rollavg_deaths','prev7_case','prev7_death']],
                 on='fips', 
                how='left')
print(county_set.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...       \
0                0.0    0.0           0.0                0.0     ...        
1                0.0    0.0           0.0                2.0     ...        
2              965.0

In [16]:
#Create indicators for rolling averages
county_set['roll7case_diff'] = ((county_set['cur_7rollavg_cases']-county_set['prev_7rollavg_cases'])/county_set['prev_7rollavg_cases'])
county_set['roll7death_diff'] = ((county_set['cur_7rollavg_deaths']-county_set['prev_7rollavg_deaths'])/county_set['prev_7rollavg_deaths'])

print(county_set.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services       ...         \
0                0.0    0.0           0.0                0.0       ...          
1                0.0    0.0           0.0                2.0       ...          
2              965.0  356.0         201.0              146.0       ...          
3                0.0    0.0           0.0                0.0       ...          
4                0.0    0.0           0.0                3.0       ...          

   cur_7rollavg_cases cur_7rollavg_deaths   curr7_case curr7_death  \
0                 NaN             

In [17]:
#Write to CSV
#county_set.to_csv('CountyMAMV2_618.csv') 

# State Level Data

In [18]:
# Pull in NYTimes state data, diff, do same 7 day rolling avg counts
# pull in COVID tracking test data. calculate pos % then rolling avgs
# Manually pull in Rt, might drop this metric

In [19]:
#Bring in state level NYT Data
nytimes_state = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
state= pd.read_csv(nytimes_state)
print(state.head())

         date       state  fips  cases  deaths
0  2020-01-21  Washington    53      1       0
1  2020-01-22  Washington    53      1       0
2  2020-01-23  Washington    53      1       0
3  2020-01-24    Illinois    17      1       0
4  2020-01-24  Washington    53      1       0


In [20]:
#Create new cases field
#Sort df by Fips and Date
state['State'] = state['state']
state.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(state.head())

#Take difference between rows for new case and new death numbers
state['case_diff'] = state.cases.diff()
state['death_diff'] = state.deaths.diff()
print(state.head())

           date    state  fips  cases  deaths    State
586  2020-03-13  Alabama     1      6       0  Alabama
637  2020-03-14  Alabama     1     12       0  Alabama
689  2020-03-15  Alabama     1     23       0  Alabama
742  2020-03-16  Alabama     1     29       0  Alabama
795  2020-03-17  Alabama     1     39       0  Alabama
           date    state  fips  cases  deaths    State  case_diff  death_diff
586  2020-03-13  Alabama     1      6       0  Alabama        NaN         NaN
637  2020-03-14  Alabama     1     12       0  Alabama        6.0         0.0
689  2020-03-15  Alabama     1     23       0  Alabama       11.0         0.0
742  2020-03-16  Alabama     1     29       0  Alabama        6.0         0.0
795  2020-03-17  Alabama     1     39       0  Alabama       10.0         0.0


In [21]:
##CREATE State Prevalence measure
#Filter for past 14 days

state_range_14 = state[(state['date']>=preval_start) & (state['date']<=preval_end)]

print(state_range_14.head())

            date    state  fips  cases  deaths    State  case_diff  death_diff
5734  2020-06-15  Alabama     1  26272     774  Alabama      657.0         1.0
5789  2020-06-16  Alabama     1  26912     785  Alabama      640.0        11.0
5844  2020-06-17  Alabama     1  27312     790  Alabama      400.0         5.0
5899  2020-06-18  Alabama     1  28206     810  Alabama      894.0        20.0
5954  2020-06-19  Alabama     1  29002     822  Alabama      796.0        12.0


In [22]:
#create state group for prevalence

state_prevalence_grouped = state_range_14.groupby(['State'], as_index=False).sum()

state_prevalence_grouped['state_active'] = state_prevalence_grouped['case_diff']
print(state_prevalence_grouped.head())

        State  fips    cases  deaths  case_diff  death_diff  state_active
0     Alabama    14   428802   11895     9826.0       146.0        9826.0
1      Alaska    28    12344     146      326.0         2.0         326.0
2     Arizona    56   758428   19416    38008.0       400.0       38008.0
3    Arkansas    70   225042    3154     7317.0        85.0        7317.0
4  California    84  2578209   77848    64002.0       848.0       64002.0


In [24]:
#Take Current and previous rolling averages

state_range_current7 = state[(state['date']>=curr_start) & (state['date']<=curr_end)]
state_range_prev7 = state[(state['date']>=prev_start) & (state['date']<=prev_end)]

print(state_range_current7.head(8))

            date    state  fips  cases  deaths    State  case_diff  death_diff
6119  2020-06-22  Alabama     1  30454     841  Alabama      433.0         2.0
6174  2020-06-23  Alabama     1  31097     864  Alabama      643.0        23.0
6229  2020-06-24  Alabama     1  32064     891  Alabama      967.0        27.0
6284  2020-06-25  Alabama     1  33206     896  Alabama     1142.0         5.0
6339  2020-06-26  Alabama     1  34183     907  Alabama      977.0        11.0
6394  2020-06-27  Alabama     1  35083     919  Alabama      900.0        12.0
6449  2020-06-28  Alabama     1  35441     919  Alabama      358.0         0.0
6120  2020-06-22   Alaska     2    869      10   Alaska       10.0         0.0


In [25]:
#Groupby Means
#Group by with mean
state_current_grouped = state_range_current7.groupby(['State'],as_index=False).mean()
state_previous_grouped = state_range_prev7.groupby(['State'], as_index=False).mean()

state_current_grouped['state_curr7_case'] = state_current_grouped['case_diff']
state_previous_grouped['state_prev7_case'] = state_previous_grouped['case_diff']
state_current_grouped['state_curr7_death'] = state_current_grouped['death_diff']
state_previous_grouped['state_prev7_death'] = state_previous_grouped['death_diff']

print(state_current_grouped.head())
print(state_previous_grouped.head())

        State  fips          cases       deaths    case_diff  death_diff  \
0     Alabama   1.0   33075.428571   891.000000   774.285714   11.428571   
1      Alaska   2.0     965.571429    10.857143    28.714286    0.285714   
2     Arizona   4.0   63922.428571  1489.428571  3042.142857   34.857143   
3    Arkansas   5.0   18009.428571   245.142857   608.142857    5.571429   
4  California   6.0  201199.428571  5778.142857  5449.714286   60.000000   

   state_curr7_case  state_curr7_death  
0        774.285714          11.428571  
1         28.714286           0.285714  
2       3042.142857          34.857143  
3        608.142857           5.571429  
4       5449.714286          60.000000  
        State  fips          cases       deaths    case_diff  death_diff  \
0     Alabama   1.0   28182.000000   808.285714   629.428571    9.428571   
1      Alaska   2.0     797.857143    10.000000    17.857143    0.000000   
2     Arizona   4.0   44424.428571  1284.285714  2387.571429   22.285

In [26]:
#Merge both sets with base file

state_active= pd.merge(county_set,
                 state_prevalence_grouped[['State','state_active',
                                       ]],
                 on='State', 
                how='left')
#print(state_current_w_pop.head(10))

state_current_w_pop = pd.merge(state_active,
                 state_current_grouped[['State','state_curr7_case',
                                       'state_curr7_death']],
                 on='State', 
                how='left')
print(state_current_w_pop.head(10))

county_state_df = pd.merge(state_current_w_pop,
                 state_previous_grouped[['State','state_prev7_case','state_prev7_death']],
                 on='State', 
                how='left')
#print(state_previous_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
0                0.0    0.0           0.0                0.0   
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.

In [27]:
#Create state rolling avg indicator
#####NEED TO ADD IN STATE POP to create per 100k 

county_state_df['state_curr7_case100k'] = ((county_state_df['state_curr7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_curr7_death100k'] = ((county_state_df['state_curr7_death']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_case100k'] = ((county_state_df['state_prev7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_death100k'] = ((county_state_df['state_prev7_death']/county_state_df['state_pop'])*100000)


county_state_df['state_roll7case_diff'] = ((county_state_df['state_curr7_case100k']-county_state_df['state_prev7_case100k'])/county_state_df['state_prev7_case100k'])
county_state_df['state_roll7death_diff'] = ((county_state_df['state_curr7_death100k']-county_state_df['state_prev7_death100k'])/county_state_df['state_prev7_death100k'])
county_state_df['state_prevalence_per100k'] = ((county_state_df['state_active']/county_state_df['state_pop'])*100000)

print(county_state_df.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
0                0.0    0.0           0.0                0.0   
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.0   
3                0.0    0.0           0.0                0.0   
4                0.0    0.0           0.0                3.0   

             ...             state_curr7_death state_prev7_case  \
0            ...                     36.142857       658.000000   
1            ...                     60.000000      3693.428571   
2    

# COVID TRACKING PROJ DATA

In [28]:
#########################################
#Bring in COVID TRACKING data
url_all = "https://covidtracking.com/api/v1/states/daily.csv"
testing_all = pd.read_csv(url_all)
testing = testing_all[['date','state','positiveIncrease','negativeIncrease', 'totalTestResultsIncrease']]
testing['state_code'] = testing['state']

print(testing.head())
print(testing.dtypes)

       date state  positiveIncrease  negativeIncrease  \
0  20200628    AK                29              2690   
1  20200628    AL               358              1896   
2  20200628    AR               570              6687   
3  20200628    AS                 0                 0   
4  20200628    AZ              3857             11292   

   totalTestResultsIncrease state_code  
0                      2719         AK  
1                      2254         AL  
2                      7257         AR  
3                         0         AS  
4                     15149         AZ  
date                         int64
state                       object
positiveIncrease             int64
negativeIncrease             int64
totalTestResultsIncrease     int64
state_code                  object
dtype: object


C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
curr_testing = testing[(testing['date']>=curr_test_end) & (testing['date']<=curr_test_start)]
curr_testing['curr_pos_%'] = (curr_testing['positiveIncrease']/curr_testing['totalTestResultsIncrease'])
print(curr_testing.head(10))

prev_testing = testing[(testing['date']>=prev_test_end) & (testing['date']<=prev_test_start)]
prev_testing['prev_pos_%'] = (prev_testing['positiveIncrease']/prev_testing['totalTestResultsIncrease'])
print(prev_testing.head())

       date state  positiveIncrease  negativeIncrease  \
0  20200628    AK                29              2690   
1  20200628    AL               358              1896   
2  20200628    AR               570              6687   
3  20200628    AS                 0                 0   
4  20200628    AZ              3857             11292   
5  20200628    CA              4810             88832   
6  20200628    CO               226              5902   
7  20200628    CT                97              8978   
8  20200628    DC                32              3829   
9  20200628    DE               135              1949   

   totalTestResultsIncrease state_code  curr_pos_%  
0                      2719         AK    0.010666  
1                      2254         AL    0.158829  
2                      7257         AR    0.078545  
3                         0         AS         NaN  
4                     15149         AZ    0.254604  
5                     93642         CA    0.051366  
6

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:

#Group by with mean
curr_test_grouped = curr_testing.groupby(['state_code'],as_index=False).mean()
prev_test_grouped = prev_testing.groupby(['state_code'], as_index=False).mean()

print(curr_test_grouped.head())
print(prev_test_grouped.head())

  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200625.0         18.285714       2874.428571   
1         AL  20200625.0        774.285714       5168.857143   
2         AR  20200625.0        595.428571       6402.857143   
3         AS  20200625.0          0.000000         74.571429   
4         AZ  20200625.0       3074.000000       9804.857143   

   totalTestResultsIncrease  curr_pos_%  
0               2892.714286    0.006533  
1               5943.142857    0.138085  
2               6998.285714    0.090052  
3                 74.571429    0.000000  
4              12878.857143    0.239803  
  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200618.0         13.428571       2070.000000   
1         AL  20200618.0        629.428571       6116.714286   
2         AR  20200618.0        377.285714       6086.285714   
3         AS  20200618.0          0.000000          0.000000   
4         AZ  20200618.0       2385.571429 

In [31]:
#Merge to County Set

curr_merge = pd.merge(county_state_df,
                 curr_test_grouped[['state_code','curr_pos_%']],
                 on='state_code', 
                how='left')
print(curr_merge.head())

mam_v2 = pd.merge(curr_merge,
                 prev_test_grouped[['state_code','prev_pos_%']],
                 on='state_code', 
                how='left')


mam_v2['testing_diff'] = (mam_v2['curr_pos_%'] - mam_v2['prev_pos_%'])
print(mam_v2.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...      \
0                0.0    0.0           0.0                0.0     ...       
1                0.0    0.0           0.0                2.0     ...       
2              965.0  356.0         201.0              146.0     ...       
3                0.0    0.0           0.0                0.0     ...       
4                0.0    0.0           0.0                3.0     ...       

   state_prev7_case state_prev7_death state_curr7_case100k  \
0        658.000000         39.857143             3.262971   
1       36

In [32]:
####################################################

####### CREATING INDICATOR/SCORES SATURDAY NIGHT!##

county_filtered = mam_v2[(mam_v2['active_cases_100k']>=0)]


#print(county_filtered.active_cases_100k.describe())
#print(county_filtered.roll7case_diff.describe())
#print(county_filtered.roll7death_diff.describe())
#print(county_filtered.state_roll7case_diff.describe())
print(county_filtered.state_prevalence_per100k.describe())


count    2868.000000
mean      124.676237
std        74.718422
min        10.727216
25%        74.235672
50%        95.142967
75%       169.423613
max       515.118668
Name: state_prevalence_per100k, dtype: float64


In [33]:
#Updated 6/25

def active(n):
    if 0<=n<=49.48: score = 0
    elif 49.49<=n<=114.05: score = 1
    elif 114.06<=n<=100000: score = 2
    else: score = 0
    return(score)



def roll(n):
    if -1<=n<=-0.10: score = 0
    elif -0.10<=n<=0.10: score = 1
    elif 0.11<=n<=100: score = 2
    else: score = 0
    return(score)

def fatal(n):
    if n>=0.05: score = 4
    else: score = 0
    return(score)

def test(n):
    if -2<=n<=-.01: score = 0
    elif -.011111<=n<=.01: score = 1
    elif 0.01111 <=n<= 100: score =2
    else: score = 0
    return(score)

def test7(n):
    if -2<=n<=-.1: score = 0
    elif 0.1 <=n<= 100: score =1
    else: score = 0
    return(score)

def curr7(n):
    if -2<=n<=-9.999: score = 0
    elif 10 <=n<= 100000: score =1
    else: score = 0
    return(score)

def testch(n):
    if 0<=n<=0.1: score = 'Decreasing'
    elif 1 <=n<= 1.9: score = 'Steady'
    else: score = 'Increasing'
    return(score)

county_filtered['County_Active_100k_Score'] = county_filtered['active_cases_100k'].apply(active)
county_filtered['State_Active_100k_Score'] = county_filtered['state_prevalence_per100k'].apply(active)
county_filtered['County_Roll_Score'] = county_filtered['roll7case_diff'].apply(roll)
county_filtered['State_Roll_Score'] = county_filtered['state_roll7case_diff'].apply(roll)
county_filtered['County_Fatality_Roll_Score'] = county_filtered['roll7death_diff'].apply(fatal)
county_filtered['State_Fatality_Roll_Score'] = county_filtered['state_roll7death_diff'].apply(fatal)
county_filtered['State_Testing_Score'] = county_filtered['testing_diff'].apply(test)
county_filtered['State_7Day_Pos_Test_Avg'] = county_filtered['curr_pos_%'].apply(test7)
county_filtered['State_7Day_New_Case_Rolling_Avg'] = county_filtered['state_curr7_case100k'].apply(curr7)


print(county_filtered.head())

    fips       County       State state_code Level 6 NBCU Flag  \
1  06037  Los Angeles  California         CA     NaN         Y   
2  17031         Cook    Illinois         IL     NaN         Y   
3  48195     Hansford       Texas         TX     NaN       NaN   
4  04013     Maricopa     Arizona         AZ     NaN         Y   
5  06073    San Diego  California         CA     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.0   
3                0.0    0.0           0.0                0.0   
4                0.0    0.0           0.0                3.0   
5                0.0    0.0           0.0                0.0   

                ...                 testing_diff County_Active_100k_Score  \
1               ...                     0.010897                        2   
2               ...                     0.001096                

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [34]:
county_filtered['State_Composite_Score'] =  (county_filtered['State_Active_100k_Score'] + county_filtered['State_Roll_Score']
                                             + county_filtered['State_Testing_Score']+county_filtered['State_7Day_New_Case_Rolling_Avg']
                                             +county_filtered['State_7Day_Pos_Test_Avg'])

county_filtered['County_Composite_Score'] = (county_filtered['County_Roll_Score'] + county_filtered['County_Active_100k_Score'])



C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [35]:
def county(n):
    if 0<=n<=.99: score = "Minimal"
    elif 0.1<=n<=1: score = "Low"
    elif 1<n<=2: score = "Moderate"
    elif 2<n<=3: score = "Severe"
    elif 4<=n<= 100: score = "Extreme"
    else: score = 0
    return(score)


def state(n):
    if 0<=n<=.99: score = "Minimal"
    elif 0.1<=n<=1: score = "Low"
    elif 1<=n<=3: score = "Moderate"
    elif 4<=n<=6: score = "Severe"
    elif 7 <=n<= 100: score = "Extreme"
    else: score = 0
    return(score)


county_filtered['County_Level'] = county_filtered['County_Composite_Score'].apply(county)
county_filtered['State_Level'] = county_filtered['State_Composite_Score'].apply(state)
print(county_filtered.columns)



Index(['fips', 'County', 'State', 'state_code', 'Level 6', 'NBCU Flag',
       'Tech Ops EE Count', 'CAE', 'Cable Stores', 'Business Services',
       'Total CCC', 'Geo Area', '%Retail Closed in County Updated 5/26',
       'Division', 'Region', 'pop', 'state_pop', 'case_diff',
       'active_cases_100k', 'cur_7rollavg_cases', 'cur_7rollavg_deaths',
       'curr7_case', 'curr7_death', 'prev_7rollavg_cases',
       'prev_7rollavg_deaths', 'prev7_case', 'prev7_death', 'roll7case_diff',
       'roll7death_diff', 'state_active', 'state_curr7_case',
       'state_curr7_death', 'state_prev7_case', 'state_prev7_death',
       'state_curr7_case100k', 'state_curr7_death100k', 'state_prev7_case100k',
       'state_prev7_death100k', 'state_roll7case_diff',
       'state_roll7death_diff', 'state_prevalence_per100k', 'curr_pos_%',
       'prev_pos_%', 'testing_diff', 'County_Active_100k_Score',
       'State_Active_100k_Score', 'County_Roll_Score', 'State_Roll_Score',
       'County_Fatality_Roll_S

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [45]:
county_fields = county_filtered[['fips','County','State','County_Level', 'State_Level', 'County_Composite_Score','roll7case_diff','active_cases_100k',
                                  'state_prevalence_per100k',
                                 'state_roll7case_diff','State_Testing_Score','State_7Day_Pos_Test_Avg','State_7Day_New_Case_Rolling_Avg',
                                 'Total CCC','CAE','Cable Stores','Business Services', 'Tech Ops EE Count']]

print(county_fields.head())
county_fields.to_csv(file) 

    fips       County       State County_Level State_Level  \
1  06037  Los Angeles  California      Extreme      Severe   
2  17031         Cook    Illinois       Severe      Severe   
3  48195     Hansford       Texas      Extreme     Extreme   
4  04013     Maricopa     Arizona      Extreme     Extreme   
5  06073    San Diego  California      Extreme      Severe   

   County_Composite_Score  roll7case_diff  active_cases_100k  \
1                       4        0.396763         247.790964   
2                       3        0.135183          94.151080   
3                       4        1.500000         129.653640   
4                       4        0.295526         585.676149   
5                       4        0.875923         116.645149   

   state_prevalence_per100k  state_roll7case_diff  State_Testing_Score  \
1                160.255443              0.475516                    0   
2                 74.235672              0.119053                    1   
3                212

# Create State Only File 

In [37]:
#Merge state data with state base file 

state_active= pd.merge(state_pop_df,
                 state_prevalence_grouped[['State','state_active',
                                       ]],
                 on='State', 
                how='left')


state_current_w_pop = pd.merge(state_active,
                 state_current_grouped[['State','state_curr7_case',
                                       'state_curr7_death']],
                 on='State', 
                how='left')


state_df = pd.merge(state_current_w_pop,
                 state_previous_grouped[['State','state_prev7_case','state_prev7_death']],
                 on='State', 
                how='left')
print(state_df.head(10))

                  State state_code  Comcast Employees  state_pop   CAE  \
0               Alabama         AL                407    4908621    65   
1                Alaska         AK                  0    3038999     0   
2               Arizona         AZ                744    7378494   237   
3              Arkansas         AR                117    3038999     1   
4            California         CA               5022   39937489     0   
5              Colorado         CO               8452    5845526  1206   
6           Connecticut         CT               1270    3563077   306   
7              Delaware         DE               1132     982895   499   
8  District of Columbia         DC                360     720687     0   
9               Florida         FL               7451   21992985  2123   

   Cable Stores  Business Services  Tech Ops  comcast_state  state_active  \
0            40                  9       178              1        9826.0   
1             0                

In [38]:
#Create State Level metrics

state_df['state_curr7_case100k'] = ((state_df['state_curr7_case']/state_df['state_pop'])*100000)
state_df['state_curr7_death100k'] = ((state_df['state_curr7_death']/state_df['state_pop'])*100000)
state_df['state_prev7_case100k'] = ((state_df['state_prev7_case']/state_df['state_pop'])*100000)
state_df['state_prev7_death100k'] = ((state_df['state_prev7_death']/state_df['state_pop'])*100000)


state_df['state_roll7case_diff'] = ((state_df['state_curr7_case100k']-state_df['state_prev7_case100k'])/state_df['state_prev7_case100k'])
state_df['state_roll7death_diff'] = ((state_df['state_curr7_death100k']-state_df['state_prev7_death100k'])/state_df['state_prev7_death100k'])
state_df['state_prevalence_per100k'] = ((state_df['state_active']/state_df['state_pop'])*100000)

print(state_df.head())

        State state_code  Comcast Employees  state_pop  CAE  Cable Stores  \
0     Alabama         AL                407    4908621   65            40   
1      Alaska         AK                  0    3038999    0             0   
2     Arizona         AZ                744    7378494  237            17   
3    Arkansas         AR                117    3038999    1            13   
4  California         CA               5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1        9826.0   
1                  0         0              0         326.0   
2                  5        87              1       38008.0   
3                  7        64              1        7317.0   
4                219      1997              1       64002.0   

             ...             state_curr7_death  state_prev7_case  \
0            ...                     11.428571        629.428571   
1            ...      

In [39]:
# Merge in Covid Tracking Data


curr_merge = pd.merge(state_df,
                 curr_test_grouped[['state_code','curr_pos_%']],
                 on='state_code', 
                how='left')

state_mam_v2 = pd.merge(curr_merge,
                 prev_test_grouped[['state_code','prev_pos_%']],
                 on='state_code', 
                how='left')


state_mam_v2['testing_diff'] = (state_mam_v2['curr_pos_%'] - state_mam_v2['prev_pos_%'])
print(state_mam_v2.head())

        State state_code  Comcast Employees  state_pop  CAE  Cable Stores  \
0     Alabama         AL                407    4908621   65            40   
1      Alaska         AK                  0    3038999    0             0   
2     Arizona         AZ                744    7378494  237            17   
3    Arkansas         AR                117    3038999    1            13   
4  California         CA               5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active      ...       \
0                  9       178              1        9826.0      ...        
1                  0         0              0         326.0      ...        
2                  5        87              1       38008.0      ...        
3                  7        64              1        7317.0      ...        
4                219      1997              1       64002.0      ...        

   state_curr7_case100k  state_curr7_death100k  state_prev7_case100k  \
0 

In [40]:
# State Level Scores


state_mam_v2['State_Active_100k_Score'] = state_mam_v2['state_prevalence_per100k'].apply(active)
state_mam_v2['State_Roll_Score'] = state_mam_v2['state_roll7case_diff'].apply(roll)
state_mam_v2['State_Fatality_Roll_Score'] = state_mam_v2['state_roll7death_diff'].apply(fatal)
state_mam_v2['State_Testing_Score'] = state_mam_v2['testing_diff'].apply(test)
state_mam_v2['State_7Day_Pos_Test_Avg'] = state_mam_v2['curr_pos_%'].apply(test7)
state_mam_v2['State_7Day_New_Case_Rolling_Avg'] = state_mam_v2['state_curr7_case100k'].apply(curr7)

print(state_mam_v2.head())

        State state_code  Comcast Employees  state_pop  CAE  Cable Stores  \
0     Alabama         AL                407    4908621   65            40   
1      Alaska         AK                  0    3038999    0             0   
2     Arizona         AZ                744    7378494  237            17   
3    Arkansas         AR                117    3038999    1            13   
4  California         CA               5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1        9826.0   
1                  0         0              0         326.0   
2                  5        87              1       38008.0   
3                  7        64              1        7317.0   
4                219      1997              1       64002.0   

                ...                 state_prevalence_per100k  curr_pos_%  \
0               ...                               200.178421    0.138085   
1     

In [41]:
#Create final dataset

state_mam_v2['State Composite Score'] =  (state_mam_v2['State_Active_100k_Score'] + state_mam_v2['State_Roll_Score']
                                             + state_mam_v2['State_Testing_Score']+state_mam_v2['State_7Day_New_Case_Rolling_Avg']
                                             +state_mam_v2['State_7Day_Pos_Test_Avg'])

state_mam_v2['State Classification'] = state_mam_v2['State Composite Score'].apply(state)
state_mam_v2['Test_Change_Classification'] = state_mam_v2['State_Testing_Score'].apply(testch)

final_state_df = state_mam_v2.query('comcast_state ==1')
print(final_state_df.head())
print(final_state_df.columns)

        State state_code  Comcast Employees  state_pop   CAE  Cable Stores  \
0     Alabama         AL                407    4908621    65            40   
2     Arizona         AZ                744    7378494   237            17   
3    Arkansas         AR                117    3038999     1            13   
4  California         CA               5022   39937489     0           491   
5    Colorado         CO               8452    5845526  1206           246   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1        9826.0   
2                  5        87              1       38008.0   
3                  7        64              1        7317.0   
4                219      1997              1       64002.0   
5                709       763              1        3175.0   

              ...              testing_diff  State_Active_100k_Score  \
0             ...                  0.029710                        2   
2       

In [42]:
#Export State File to CSV

final_state_df2 = final_state_df[['State', 'State Classification', 'State Composite Score', 'state_prevalence_per100k',
                                 'state_roll7case_diff','Test_Change_Classification','State_7Day_Pos_Test_Avg','State_7Day_New_Case_Rolling_Avg',
                                 'Comcast Employees','CAE','Cable Stores','Business Services', 'Tech Ops']]

                                  
print(final_state_df2.head())
#final_state_df.to_csv(state_file) 

        State State Classification  State Composite Score  \
0     Alabama              Extreme                      8   
2     Arizona              Extreme                      8   
3    Arkansas               Severe                      6   
4  California               Severe                      5   
5    Colorado               Severe                      4   

   state_prevalence_per100k  state_roll7case_diff Test_Change_Classification  \
0                200.178421              0.230141                 Increasing   
2                515.118668              0.274158                 Increasing   
3                240.770069              0.391176                     Steady   
4                160.255443              0.475516                 Decreasing   
5                 54.315044              0.253371                     Steady   

   State_7Day_Pos_Test_Avg  State_7Day_New_Case_Rolling_Avg  \
0                        1                                1   
2                        1

In [43]:
#################FOR SCORE CREATION#########


state_mam_v3 = state_mam_v2[['State', 'State Classification', 'State Composite Score', 'state_prevalence_per100k',
                                 'state_roll7case_diff','Test_Change_Classification','State_7Day_Pos_Test_Avg','State_7Day_New_Case_Rolling_Avg',
                                 'Comcast Employees','CAE','Cable Stores','Business Services', 'Tech Ops','testing_diff','state_curr7_case100k','curr_pos_%','state_prev7_case100k']]



In [44]:
state_mam_v3.to_csv('for_scoring_629.csv') 